In [ ]:
#Predicting_Model

def paintConfusion_digit(lmr_matrix,classes):
    plt.imshow(lmr_matrix,interpolation='nearest',cmap=plt.cm.Blues)
   # plt.imshow(lmr_matrix,interpolation='nearest')
    plt.title('confusion matrix')
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=45)
    plt.yticks(tick_marks,classes)
    plt.xlabel('Predict label')
    plt.ylabel('True label')
    fmt='d'
    thresh=lmr_matrix.max()/2.
    for i,j in itertools.product(range(lmr_matrix.shape[0]),range(lmr_matrix.shape[1])):
        plt.text(j, i, format(lmr_matrix[i, j], fmt),
                     horizontalalignment="center",
                     color="black" if lmr_matrix[i, j] > thresh else "black")
    plt.tight_layout()
    plt.show()

#ROC AUC
def paintROC(y_test1,y_pred1):
    fpr,tpr,threshold = roc_curve(y_test1, y_pred1) ###real p value/ false p value
    roc_auc = auc(fpr,tpr) ###auc

    plt.figure()
    lw = 2
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='blue',
            lw=lw, label='ROC curve (area = %0.4f)' % roc_auc) ###
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    
import pandas as pd
import numpy as np
import os
import xlrd
import xlsxwriter
import xlwt
from collections import Counter
import matplotlib.pyplot as plt
import itertools
import operator
from sklearn.metrics import classification_report

#import data
data=pd.read_csv(r'E:\Data_test\car_insurance_claim.csv')

#drop missing value rows
data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True) 
x=data
kid=np.array(x['KIDSDRIV'])
age=np.array(x['AGE'])
homekid=np.array(x['HOMEKIDS'])
yoj=np.array(x['YOJ'])
income=np.array(x['INCOME'])
val=np.array(x['HOME_VAL'])
travtime=np.array(x['TRAVTIME'])
oldclaim=np.array(x['OLDCLAIM'])
fre=np.array(x['CLM_FREQ'])
mvr=np.array(x['MVR_PTS'])
amt=np.array(x['CLM_AMT'])
car_age=np.array(x['CAR_AGE'])
bluebook=np.array(x['BLUEBOOK'])

parent=np.array(pd.get_dummies(x['PARENT1']),dtype=np.float64)
mstatus=np.array(pd.get_dummies(x['MSTATUS']),dtype=np.float64)
edu=np.array(pd.get_dummies(x['EDUCATION']),dtype=np.float64)
sex=np.array(pd.get_dummies(x['GENDER']),dtype=np.float64)
occupation=np.array(pd.get_dummies(x['OCCUPATION']),dtype=np.float64)
car_use=np.array(pd.get_dummies(x['CAR_USE']),dtype=np.float64)
car_type=np.array(pd.get_dummies(x['CAR_TYPE']),dtype=np.float64)
tif=np.array(pd.get_dummies(x['TIF']),dtype=np.float64)
red=np.array(pd.get_dummies(x['RED_CAR']),dtype=np.float64)
revoked=np.array(pd.get_dummies(x['REVOKED']),dtype=np.float64)

x_initial=np.c_[kid,age,homekid,yoj,income,val,travtime,oldclaim,fre,mvr,amt,car_age,bluebook,parent,mstatus,
               edu,sex,occupation,car_use,car_type,tif,red,revoked]
y=np.array(x['CLAIM_FLAG'])

#Standardization
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()
x_initial = Imputer().fit_transform(x_initial)
X=scaler.fit_transform(x_initial)


from sklearn.model_selection import StratifiedKFold  

cv_number=5# parameter

skf= StratifiedKFold(n_splits=cv_number) # x-fold cv 
skf.get_n_splits(X,y) 
print("CV method:StratifiedKFlod  Split number: ",cv_number)

from sklearn.metrics import confusion_matrix
import time
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc

ac_test=[]
ac_train=[]
term=1

for train_index, test_index in skf.split(X, y):  
    print("Start:   current: ",term,"  ","Total: ",cv_number) 
    time_start=time.time()
    X_train,X_test = X[train_index], X[test_index]  
    y_train,y_test = y[train_index], y[test_index]
    
    #train model on different training base
    #clf=svm.SVC(C=2**(c_exp),kernel='rbf', gamma=2**(gamma_exp))# svm with radical basis function
#    clf=svm.SVC()#修改这行
    #clf=RandomForestClassifier(min_samples_leaf=msl,n_estimators=n_e,n_jobs=-1,random_state=3)
#    clf=RandomForestClassifier()#修改这行
  #  clf = GradientBoostingClassifier(n_estimators=10, random_state=10, subsample=0.6, max_depth=7,
                #                  min_samples_split=900)
 #   clf=GradientBoostingClassifier()
#    clf=xgb()
    xgb_params = {'booster':'gbtree','objective': 'binary:logistic', "eta": 0.01, "max_depth": 5,  "silent": 0,"colsample_bytree":0.7}
    num_rounds = 100
  #  clf = xgb.XGBClassifier()
    clf=lgb.LGBMClassifier()
 #   clf = LogisticRegression()
   # clf=MLPClassifier()
    
    clf.fit(X_train,y_train) # fit the model
  #  y_pred_test=clf.predict(X_test) #predict y based on X_test
    
    #output prob
    y_pred_test=clf.predict_proba(X_test)[:,1] 
    y_pred_train=clf.predict(X_train) #predict y based on X_train
#    ac_test.append(metrics.accuracy_score(y_test, y_pred_test))
    ac_train.append(metrics.accuracy_score(y_train,y_pred_train))
    time_end=time.time()
    term+=1
#    print("Time: ",round(time_end-time_start,2),
 #         " ac_train: ",round(metrics.accuracy_score(y_train,y_pred_train),4),
  #        " ac_test: ",round(metrics.accuracy_score(y_test, y_pred_test),4))
   
    #cm = confusion_matrix(y_test,y_pred_test)

  #Draw Confusion matrix  
  #  paintConfusion_digit(cm,[0,1])
 #   print(classification_report(y_test, y_pred_test))
    paintROC(y_test,y_pred_test)
    
ac_test=np.array(ac_test)
ac_train=np.array(ac_train)
print("Mean Accuracy of Train Set: ",round(ac_train.mean(),4),
     " Mean Accuracy of the Test Set:",round(ac_test.mean(),4))
#print("Parameters:")
#print("Regularization: 2**{}".format(c_exp),"  Gamma of RBF: 2**{} ".format(gamma_exp))
